# Particle Swarm Optimisation

This notebook showcases how to use the built-in Particle Swarm Optimisation (PSO) algorithm.

In [1]:
using EvoLP
using Statistics
using OrderedCollections

For this example, we will use the Michalewicz function:

In [2]:
@doc michalewicz

```
michalewicz(x; m=10)
```

The **Michalewicz** function is a $d$-dimensional function with several steep valleys, where `m` controls the steepness. `m` is usually set at 10. For 2 dimensions, $x^* = [2.20, 1.57]$, with $f(x^*) = -1.8011$.

$$
f(x) = -\sum_{i=1}^{d}\sin(x_i) \sin^{2m}\left(\frac{ix_i^2}{\pi}\right)
$$


In this case we will use `d=2` and `m=10`, which is the default value implemented in EvoLP.

In PSO, we use _particles_. Each particle has a position and a velocity, and remembers the best position it has visited.

We can create a population of particles in multiple ways, but EvoLP provides 2 particle generators with random positions: either uniform or following a normal distribution.

Let's use the normal generator:

In [3]:
@doc normal_rand_particle_pop

```
normal_rand_particle_pop(n, μ, Σ; rng=Random.GLOBAL_RNG)
```

Generate a population of `n` [`Particle`](@ref) using a normal distribution with means `μ``and covariance`Σ`.

`μ` expects a vector of length *l* (i.e. number of dimensions) while `Σ` expects an *l x l* matrix of covariances.

# Examples

```julia
julia> normal_rand_particle_pop(3, [0, 0], [1 0; 0 1])
3-element Vector{Particle}:
 Particle([-0.6025996585348097, -1.0055548956861133], [0.0, 0.0], Inf, [-0.6025996585348097, -1.0055548956861133], Inf)
 Particle([-0.7562454555135321, 1.9490439959687778], [0.0, 0.0], Inf, [-0.7562454555135321, 1.9490439959687778], Inf)
 Particle([0.5687241357408321, -0.7406267072113427], [0.0, 0.0], Inf, [0.5687241357408321, -0.7406267072113427], Inf)
```


In [4]:
population = normal_rand_particle_pop(50, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Particle}:
 Particle([-0.1290950189447983, 0.3501210500290379], [0.0, 0.0], Inf, [-0.1290950189447983, 0.3501210500290379], Inf)
 Particle([1.4975284747195907, -0.29564202794924843], [0.0, 0.0], Inf, [1.4975284747195907, -0.29564202794924843], Inf)
 Particle([-0.9245723438152627, 0.2860461202845279], [0.0, 0.0], Inf, [-0.9245723438152627, 0.2860461202845279], Inf)

Let's use the `Logbook` to save information about each iteration of the run:

In [5]:
@doc Logbook

```
Logbook()
Logbook(S::LittleDict)
```

A log for statistics intended for use on every iteration of an algorithm. The logbook is constructed from a `LittleDict` ordered dictionary which maps stat names (strings) to callables, such that *statname* $i$ can be computed from *callable* $i$.

The resulting `Logbook` contains:

  * `S::LittleDict`: The ordered dict of stat names and callables
  * `records::AbstractVector`: A vector of NamedTuples where each field is a statistic.

If no argument is passed, the logbook is constructed with a set of commonly statistics such as minimum, mean, median, maximum and standard deviation; in that order.


In [6]:
statnames = ["avg_fit", "median_fit", "best_fit"]
callables = [mean, median, minimum]

thedict = LittleDict(statnames, callables)
logbook = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("avg_fit" => Statistics.mean, "median_fit" => Statistics.median, "best_fit" => minimum), NamedTuple{(:avg_fit, :median_fit, :best_fit)}[])

We can now use the built-in algorithm:

In [7]:
@doc PSO

```
PSO(f, population, k_max; w=1, c1=1, c2=1)
PSO(logger::Logbook, f, population, k_max; w=1, c1=1, c2=1)
```

## Arguments

  * `f::Function`: Objective function to **minimise**.
  * `population::Vector{Particle}`: a list of [`Particle`](@ref) individuals.
  * `k_max::Integer`: number of iterations.

## Keywords

  * `w`: inertia weight. Optional, by default 1.
  * `c1`: cognitive coefficient (own's position). Optional, by default 1.
  * `c2`: social coefficient (others' position). Optional, by default 1.

Returns a [`Result`](@ref).


In [8]:
results = PSO(logbook, michalewicz, population, 30);

The output was suppressed so that we can analyse each part of the result separately:

In [9]:
@show optimum(results)

@show optimizer(results)

@show iterations(results)
@show f_calls(results);

optimum(results) = -1.8012909805009536
optimizer(results) = 

[2.2020691613746304, 1.5706341549186813]
iterations(results) = 30
f_calls(results) = 1550


We can also take a look at the logbook's records and see how the statistics we calculated changed throughout the run:

In [14]:
for (i, I) in enumerate(logbook.records[end:-1:25])
    print("iteration: end-$(i-1) with best_pos: $(I[3]) and avg_pos: $(I[1]) \n")
end

iteration: end-0 with best_pos: -1.800603898924665 and avg_pos: -0.5390680730400352 
iteration: end-1 with best_pos: -1.8012909805009536 and avg_pos: -0.5579480641388794 
iteration: end-2 with best_pos: -1.8007014593612416 and avg_pos: -0.4999215009701753 
iteration: end-3 with best_pos: -1.8009863448728947 and avg_pos: -0.5064147295304965 
iteration: end-4 with best_pos: -1.801223803629051 and avg_pos: -0.568827195140446 
iteration: end-5 with best_pos: -1.8007807467177979 and avg_pos: -0.6327865064430491 
